In [7]:
from gurobipy import *
import numpy as np
import math as mt


In [8]:
def random_matrice_utilite(n, p):
        return np.array([[19, 6, 17, 2], [2, 11, 4, 18]])

def random_cout_projet(p):
        return np.array([40, 50, 60, 50])

def generate_w_prime():
        # vecteur w
        w = [2, 1]

        # creer le vecteur w'
        w_prime = [(w[k] - w[k+1]) for k in range(len(w)-1)]
        w_prime.append(w[-1])
        return w_prime

def generate_contrainte_matrice(n, p, u, couts_projet):
        ## les variables rk
        coef_rk = []
        for k in range(n):
                for i in range(n):
                        coef_rk.append([])
                        for j in range(n):
                                coef_rk[-1].append(0)
                        coef_rk[-1][k] = 1
        coef_rk = np.array(coef_rk)
        ## les variables bik
        coef_bik = -1 * np.identity(n**2)
        ## les variables x
        coef_xi = []
        for k in range(n**2):
                coef_xi.append([])
                for i in range(p):
                        coef_xi[-1].append(-u[k%n][i])
        coef_xi = np.array(coef_xi)
        # contrainte sur le budget
        budget = [0 for i in range(n*(n+1))]
        budget += [couts_projet[i] for i in range(p)]
        budget = np.array(budget)
        # concatenation des matrices rk, bik, xk 
        result_concat = np.concatenate(
                (coef_rk, np.concatenate((coef_bik, coef_xi),axis=1)),
                axis=1)

        # ajout de la contrainte sur les budgets
        constrainte = np.vstack([result_concat, budget])
        return constrainte
def second_membre_contraintes(n, budget_limit):
        second_membre = [0 for _ in range(n**2)]
        second_membre.append(budget_limit)
        second_membre = np.array(second_membre)
        return second_membre

def coef_func_objectif(w_prime, n, p):
        ## coefficient fonction objectif
        c = [(i+1) * w_prime[i] for i in range(n)] # coef des r_k
        # coef des b_ik
        for i in range(n) :
                for j in range(n) :
                        c.append(-1*w_prime[i])

        c += [0 for i in range(p)] # ajout des coef de x
        c = np.array(c)
        print(c)
        return c

def declare_variables(m, n, p):
        x = []
        for i in range(n):
                x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="r%d" % (i+1)))
        for i in range (n):
                for j in range(n):
                        x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="b%d%d" % ((j+1,(i+1)))))
        for j in range(p):
                x.append(m.addVar(vtype=GRB.BINARY, lb=0, name="x%d" % (j+1) ))
        return x

def definition_func_objectif(m, n, p, c, x):
        obj = LinExpr();
        obj =0
        for j in range(n*n+n+p):
                obj += c[j] * x[j]

        m.setObjective(obj,GRB.MAXIMIZE)
        return obj
        
def definition_contraintes(m, x, constrainte, second_membre, n, p):
        # Definition des contraintes
        # Definition des contraintes
        lignes = range(n*n+1)
        colonnes = range(n*n+n+p)
        print (lignes, colonnes)
        for i in lignes:
                m.addConstr(quicksum(constrainte[i][j]*x[j] for j in colonnes) <= second_membre[i], "Contrainte%d" % i)
        
def display_vars_value(m):
        print("")                
        print('Solution optimale:')
        mvars = m.getVars()
        names = m.getAttr('VarName', mvars)
        values = m.getAttr('X', mvars)
        result = dict(zip(names, values))
        for key in result:
                print(key,'=',result[key])
        print("")
        print('Valeur de la fonction objectif :', m.objVal)

In [9]:
def experience(n, p):
        matrice_utilite = random_matrice_utilite(n, p)
        couts_projet = random_cout_projet(p)
        w_prime = generate_w_prime()
        contrainte = generate_contrainte_matrice(n, p, matrice_utilite, couts_projet)
        second_membre = second_membre_contraintes(n, 100)
        coef_fonction_objectif = coef_func_objectif(w_prime, n, p)
        m = Model("Experimentation 1.3")
        x = declare_variables(m, n, p)
        m.update()
        obj = definition_func_objectif(m, n, p, coef_fonction_objectif, x)
        definition_contraintes(m, x, contrainte, second_membre, n, p)
        m.optimize()
        display_vars_value(m)
        

experience(2, 4)        

[ 9  2 -9 -9 -1 -1  0  0  0  0]
range(0, 5) range(0, 10)
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 10 columns and 28 nonzeros
Model fingerprint: 0xbc42fd8e
Variable types: 6 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [1e+00, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]
Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 5 rows, 10 columns, 28 nonzeros
Variable types: 6 continuous, 4 integer (4 binary)

Root relaxation: objective 2.442000e+02, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  244.200